In [1]:
library(ArchR)
library(org.Ss.eg.db)
library(SuscrofaTxdb.11.108.july)
library(Repitools)


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

In [2]:
options(repr.plot.width = 18, repr.plot.height = 17, repr.plot.pointsize = 24)

In [3]:
addArchRThreads(2)

Setting default number of Parallel threads to 2.



In [4]:
load(file = "/home/adufour/work/rds_storage/omics/archr_all_v7_embryo.RData")

In [5]:
peak_grange <- archrproj_sub@peakSet

In [6]:
peak_grange$peakName <- peak_grange %>% {paste0(seqnames(.), "_", start(.), "_", end(.))}

In [7]:
names(peak_grange) <- NULL

In [8]:
peak_DF <- annoGR2DF(peak_grange)

In [10]:
markersPeaks <- getMarkerFeatures(
    ArchRProj = archrproj_sub, 
    useMatrix = "PeakMatrix", 
    groupBy = "Clusters",
    bias = c("TSSEnrichment", "log10(nFrags)"),
    testMethod = "wilcoxon"
)

ArchR logging to : ArchRLogs/ArchR-getMarkerFeatures-3e1f5e496b0dcc-Date-2024-03-15_Time-16-13-08.log
If there is an issue, please report to github with logFile!

MatrixClass = Sparse.Integer.Matrix

2024-03-15 16:13:10 : Matching Known Biases, 0.025 mins elapsed.

Warning message:
“le package ‘nabor’ a été compilé avec la version R 4.2.3”
###########
2024-03-15 16:39:20 : Completed Pairwise Tests, 26.189 mins elapsed.
###########

ArchR logging successful to : ArchRLogs/ArchR-getMarkerFeatures-3e1f5e496b0dcc-Date-2024-03-15_Time-16-13-08.log



# export DAG whole

In [11]:
scriptPath <- "/home/adufour/work/scScalpChromatin"
source(paste0(scriptPath, "/misc_helpers.R"))
source(paste0(scriptPath, "/matrix_helpers.R"))
source(paste0(scriptPath, "/plotting_config.R"))
source(paste0(scriptPath, "/archr_helpers.R"))

Warning message:
“le package ‘dplyr’ a été compilé avec la version R 4.2.3”
Warning message:
“le package ‘tidyr’ a été compilé avec la version R 4.2.3”
Warning message:
“le package ‘RcppAlgos’ a été compilé avec la version R 4.2.3”
Warning message:
“le package ‘ggrastr’ a été compilé avec la version R 4.2.3”


In [12]:
corrCutoff <- 0.5
p2gGR <- getP2G_GR(archrproj_sub, corrCutoff=corrCutoff)

In [13]:
p2geneDF <- Repitools::annoGR2DF(p2gGR)

In [14]:
markerPeaksList <- getMarkers(markersPeaks, cutOff = "FDR <= 0.1 & abs(Log2FC) >= 1")

In [25]:
markerPeaksList$C1$peakName <- paste0(markerPeaksList$C1$seqnames, "_", markerPeaksList$C1$start, "_", markerPeaksList$C1$end)
markerPeaksList$C2$peakName <- paste0(markerPeaksList$C2$seqnames, "_", markerPeaksList$C2$start, "_", markerPeaksList$C2$end)
markerPeaksList$C3$peakName <- paste0(markerPeaksList$C3$seqnames, "_", markerPeaksList$C3$start, "_", markerPeaksList$C3$end)
markerPeaksList$C4$peakName <- paste0(markerPeaksList$C4$seqnames, "_", markerPeaksList$C4$start, "_", markerPeaksList$C4$end)

In [18]:
peak_grange <- archrproj_sub@peakSet

In [19]:
peak_grange$peakName <- peak_grange %>% {paste0(seqnames(.), "_", start(.), "_", end(.))}

In [20]:
names(peak_grange) <- NULL

In [21]:
peak_DF <- Repitools::annoGR2DF(peak_grange)

In [29]:
merge_C1 <- merge(markerPeaksList$C1,p2geneDF,by="peakName", all.x=TRUE)
merge_C2 <- merge(markerPeaksList$C2,p2geneDF,by="peakName", all.x=TRUE)
merge_C3 <- merge(markerPeaksList$C3,p2geneDF,by="peakName", all.x=TRUE)
merge_C4 <- merge(markerPeaksList$C4,p2geneDF,by="peakName", all.x=TRUE)

In [30]:
write.csv(merge(merge_C1,peak_DF,by="peakName"), "/home/adufour/work/table/omics_embryo_dag_c1.csv", row.names=TRUE)
write.csv(merge(merge_C2,peak_DF,by="peakName"), "/home/adufour/work/table/omics_embryo_dag_c2.csv", row.names=TRUE)
write.csv(merge(merge_C3,peak_DF,by="peakName"), "/home/adufour/work/table/omics_embryo_dag_c3.csv", row.names=TRUE)
write.csv(merge(merge_C4,peak_DF,by="peakName"), "/home/adufour/work/table/omics_embryo_dag_c4.csv", row.names=TRUE)

In [10]:
write.csv(peak_DF, "/home/adufour/work/table/peak_DF_stemcells.csv", row.names=FALSE)